<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GAN:-theory-and-applications" data-toc-modified-id="GAN:-theory-and-applications-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GAN: theory and applications</a></span><ul class="toc-item"><li><span><a href="#Generator" data-toc-modified-id="Generator-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generator</a></span></li><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Discriminator</a></span></li><li><span><a href="#Intuitive-explaination" data-toc-modified-id="Intuitive-explaination-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Intuitive explaination</a></span></li><li><span><a href="#Non-saturating-value-function" data-toc-modified-id="Non-saturating-value-function-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Non saturating value function</a></span></li><li><span><a href="#Models-definition" data-toc-modified-id="Models-definition-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Models definition</a></span></li><li><span><a href="#Training-phase" data-toc-modified-id="Training-phase-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Training phase</a></span></li><li><span><a href="#Type-of-GANs" data-toc-modified-id="Type-of-GANs-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Type of GANs</a></span></li><li><span><a href="#Conditional-GANs" data-toc-modified-id="Conditional-GANs-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Conditional GANs</a></span></li><li><span><a href="#Applications" data-toc-modified-id="Applications-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Applications</a></span><ul class="toc-item"><li><span><a href="#Unconditional-GAN" data-toc-modified-id="Unconditional-GAN-1.9.1"><span class="toc-item-num">1.9.1&nbsp;&nbsp;</span>Unconditional GAN</a></span></li><li><span><a href="#Conditional-GAN" data-toc-modified-id="Conditional-GAN-1.9.2"><span class="toc-item-num">1.9.2&nbsp;&nbsp;</span>Conditional GAN</a></span></li></ul></li></ul></li></ul></div>

# GAN: theory and applications

*Adversarial Training (also called GAN for Generative Adversarial Networks) is most interesting idea in the last 10 years of ML* <sup>[1](#1)</sup>

GANs are a framework for the estimation of generative models via an adversarial process in which 2 models, a **discriminator** $D$ and a **generator** $G$, are trained simultaneously.

The generative model $G$ aim is to capture the data distribution, whilst the discriminative model $D$ estimates the probability that a sample came from the training data rhather then $G$.

The power of the adversarial training framework cames from the fact that both $D$ and $G$ can be a non-linear, parametric, mapping functions such as **neural networks** and all the network can be trained end to end using **gradient descent** or **gradient ascent** depening of we are interested in minimizing or maximizing an objective.

To learn a generator distribtution $p_g$ over the data **$x$** the generator builds a mapping from a **prior** noise distribution $p_z(z)$ to a data space as $G(z;\theta_g)$.

The discriminator $D(x;\theta_d)$ outputs a single scalar representing the probability that $x$ came from real data rather than $p_g$.

The original GAN framework poses this problem as a **min-max** game in which the two players ($G$ and $D$) compete one against the other.

## Generator

The generator is responsible for learning a distribution as good as a sample $Y \sim p_g$ can fool the discriminator

## Discriminator

The discriminator is responsibile for classifiyng the generated sample $D(Y) = D(G(z))$ in 1 of 2 different classes: **real** vs **fake**.

Generator and Discriminator compete against each other, playing the following zero sum min-max game with value function $V_{GAN}(D,G)$

$$ \min_G \max_D V_{GAN}(D,G) = \mathbb{E} _{x \sim p_{data}(x)}[log D(x)] + \mathbb{E}_{z \sim p_z(z)}[\log(1 - D(G(z)))] $$

## Intuitive explaination

We want to be sure to train the **discriminator** $D$ in order to correctly classify the values sampled from the real data (**maximize** $\mathbb{E}_{x\sim p_{data}(x)}[\log D(x)]$ ) and **at the same time** given a fake sample $G(z), z \sim p_z(z)$ the discriminator is expected to output a probability $D(G(z))$ close to zero, by **maximizing** $\mathbb{E}_{z \sim p_z(z)}[\log(1- D(G(z)))]$.

The **generator** instead, is trained in order to *fool the discriminator*, so it will learn to produce samples that are more and more similar to the ones sampled from the real data distribution and it will do this by **minimizing** $\mathbb{E}_{z \sim p_z(z)}[\log(1- D(G(z)))]$

Note: *the minmax game is played only in the second part of the equation, in fact when updating the generator parameters the first term has no impact*.

## Non saturating value function

As Goodfellow itself pointed out in the original GAN paper<sup>[2](#2)</sup> the previous equation may not provide sufficient gradient for $G$ to learn well. Early in learning, when the quality of the generated samples from $G$ is poor, $D$ can reject samples with high confidence because they are clearly different from the training data. In this case, $\log(1 - D(G(z)))$ **saturates**.

The proposed solution is to train $G$ to **maximize** $\log D(G(z))$ instead of minimizing $\log(1 - D(G(z)))$. This means that the 2 networks are going to optimize 2 different, but interacting, value functions (they are playing the same game, in a different manner):

$$V_{GAN}(D,G) = \begin{cases}
D: & \min_G \max_D  \mathbb{E} _{x \sim p_{data}(x)}[log D(x)] + \mathbb{E}_{z \sim p_z(z)}[\log(1 - D(G(z)))] \\
G: & \max_G \mathbb{E}_{z \sim p_z(z)}[\log(D(G(z)))]
\end{cases}$$


## Models definition

As stated above, the power of GAN is that both $G$ and $D$ can be parametrized functions. Defining $G$ and $D$ as neural networks parametrized trough the sets of parameters $\theta_g$ and $\theta_d$ allow us to exploit the power of different neural networks architectures to reach different aims.

In fact, there are **a lot** of different nerual network architectures that can be used depending on the **data type** we're working on and on the **dataset size**.

- Tuple of numbers? Fully connected (FC)
- Text? Recurrent Neural Network (RNN)
- Images? Convolutional Neural Networks (CNN)

Every macro-category listed above, has its **huge** set of different architectures (e.g.: in the CNN domain, there are hundrends of well-known different architectures like LeNet, VGG, ResNet, SqueeezeNet, ...) and GAN authors themself suggest other architectures that worked well in their experiments.

However, definining both $D$ and $G$ as parametrized funtion, allow us to train the whole GAN end to end using **gradient descent**.

## Training phase

Playing this min-max game consists in the **alternating** execution of training steps. As described in the "Non saturating value function" section, the discriminator and the generator compete against each other and they do this in turn.

In order to be computationally tractable (and to make the training process more stable) it's not used the **pure gradient descent** but instead, as we usually do in most of the case when working with ML algorithms, the training process is made using **minibatch stochastic gradient descent**.

The game follow this rules:

**Discriminator**: this steps can be repeated from 1 to $k$ times, where $k$ is an hyperparameter.

1. Sample minibatch of $m$ noise samples ${z^{(1)},\dots,z^{(m)}}$ from noiise prior $p_g(z)$
2. Sample minimatch of $m$ examples ${x^{(1)},\dots,x^{(m)}}$ from data generating distribution $p_{data}(x)$
3. **Train the discriminator** by **ascending** its stochastic gradient:
$$ \Delta_{\theta_d} \frac{1}{m} \sum_{i=1}^{m}\log D(x^{(i)}) + \log(1 - D(G(z^{(i)}) $$

**Generator**: the update is executed only once and only after the turn of the disciminator is completed

1. Sample minibatch of $m$ noise samples ${z^{(1)},\dots,z^{(m)}}$ from noiise prior $p_g(z)$
2. **Train the generator** by **ascending** its stochastic gradient (we're using the non saturating value function):

$$ \Delta_{\theta_g} \frac{1}{m} \sum_{i=1}^{m} \log(D(G(z))) $$

The gradient-based updates can use any standard optimization algorithm, since we're using neural netwokrs (hence we can use the momentum, Adam, RMSProp, ...)

The training phase (discriminator + generator) can last for an arbitrary number of training steps (or better, for an aribitrary number of epochs, that's a way better way to measure the training periods).

The train can be considered completed when the discriminator is **completely fooled** by the generator: e.g. when the value of $D$ is always 0.5 that means that the best $D$ can do is only guess, there are 2 classes and each class has equal probability (**intitive explaination**: the samples generated are very similar to the real data and the discriminator now can only guess).

## Type of GANs

Until now, the description of the GAN from the idea to the architectures passing trough the training phase has been described just like there's only one type of GAN. This is not true.

There are not only **thousands** of different GAN architectures<sup>[3](#3)</sup> but also GANs that can be trained to generate explicitly certain samples, adding a condition to the previosly described architecture.

In particular, we can classify GANs in two big families:

- Unconditional GANs
- Conditional GANs

The unconditional GANs are the type of GANs just described, while the conditional GAN are a sightly modification of the GAN formulation that allow to force the generator to take into account a condition and generate only the specified type of sample.

## Conditional GANs

The conditional GAN paper <sup>[4](#4)</sup> clearly describes the aim and the idea of conditional GANs, so I'm going to cite the paper (with some little change) in this section.

GANs can be extended to a conditional model if **both** $G$ and $D$ are conditioned on some extra information $y$. $y$ could be any kind of auxiliary information, such as class labels or data from other modalities. We can perform the conditioning by feeding $y$ into both the discriminator and generator as **additional input layer**.

In the **generator** the prior input noise $p_z(z)$ and $y$ are combined in joint hidden representation, and the adversarial training framework allows for considerable flexibility in how this hidden representation is composed.

In the **discriminator** $x$ and $y$ are presented as inputs to a discriminative function (the condition has to be presented in some hidden representation).

The objective function of a two player min-max game would be:

$$ \min_G \max_D V_{GAN}(D,G) = \mathbb{E} _{x \sim p_{data}(x|y)}[log D(x)] + \mathbb{E}_{z \sim p_z(z)}[\log(1 - D(G(z|y)))] $$

One thing to note is **how** we present the condition $y$ as input to the networks: the same representation of the condition has to be presented to both networks. Usually, this is more easy than you tought: just concat the reprensetation of the condition (the 1-hot encoded label? the semantic map? ...) to the input of $G$ and the input of $D$.


## Applications

Adverarial training (unconditional/conditional) has been used to solve a lot of different tasks, from generation of faces to domain translation, passing trough super resolution applications. In this section you'll se some of this application, just to let you know how powerful the adversarial training framework is and how infinite are the possibilities.

### Unconditional GAN

![Learning process of the MNIST dataset](images/mnist.gif)

A gif representing the evolution over time of the generator output.

### Conditional GAN

From the Conditional GAN paper, the output of the generator when trained on the MNIST dataset, where each row is conditioned on one label.

![Conditional MNIST](images/conditional_mnist.png)


From the Image-to-Image Translation with Conditional Adversarial Networks paper<sup>[5](5)</sup> (*pix2pix*), here's a clever application of conditional GAN to solve some extremely hard computer vision tasks:

![pix2pix](images/pix2pix.png)

**References**

<a id="1">[1]</a>: According to Yann LeCun answer on Quora: https://www.quora.com/What-are-some-recent-and-potentially-upcoming-breakthroughs-in-deep-learning/answer/Yann-LeCun

<a id="2">[2]</a>: Generative Adversarial Networks https://arxiv.org/pdf/1406.2661.pdf

<a id="3">[3]</a>: The GAN Zoo https://github.com/hindupuravinash/the-gan-zoo

<a id="4">[4]</a>: Conditional Generative Adversarial Nets https://arxiv.org/pdf/1411.1784.pdf

<a id="5">[5]</a>: Image-to-Image Translation with Conditional Adversarial Networks https://arxiv.org/pdf/1611.07004.pdf